# Train a binary classification model using PyTorch

We use the breast cancer wisconsin dataset to train and test this binary classification model.

In [40]:
#Put all the libraries here
import numpy as np

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

from sklearn import datasets
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split

## Load breast cancer wisconsin dataset from scikit-learn 



In [8]:
cancer = datasets.load_breast_cancer()

#Classe names
print("***********Classe names***********")
print(cancer.target_names)
#Feature names
print("***********Feature names***********")
print(cancer.feature_names)
#Data sample festure values
print("***********Data sample festure values***********")
print(cancer.data)
print("***********Data size and feature size***********")
print(cancer.data.shape)
#Data label values
print("***********Data label values***********")
print(cancer.target)

***********Classe names***********
['malignant' 'benign']
***********Feature names***********
['mean radius' 'mean texture' 'mean perimeter' 'mean area'
 'mean smoothness' 'mean compactness' 'mean concavity'
 'mean concave points' 'mean symmetry' 'mean fractal dimension'
 'radius error' 'texture error' 'perimeter error' 'area error'
 'smoothness error' 'compactness error' 'concavity error'
 'concave points error' 'symmetry error' 'fractal dimension error'
 'worst radius' 'worst texture' 'worst perimeter' 'worst area'
 'worst smoothness' 'worst compactness' 'worst concavity'
 'worst concave points' 'worst symmetry' 'worst fractal dimension']
***********Data sample festure values***********
[[1.799e+01 1.038e+01 1.228e+02 ... 2.654e-01 4.601e-01 1.189e-01]
 [2.057e+01 1.777e+01 1.329e+02 ... 1.860e-01 2.750e-01 8.902e-02]
 [1.969e+01 2.125e+01 1.300e+02 ... 2.430e-01 3.613e-01 8.758e-02]
 ...
 [1.660e+01 2.808e+01 1.083e+02 ... 1.418e-01 2.218e-01 7.820e-02]
 [2.060e+01 2.933e+01 1.401e+

## Convert the data to PyTorch tensors 

In [18]:
X, y = cancer.data, cancer.target

#Split the data into two sets: 80% for training and 20% for testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# Create a TensorDataset for training and testing, respectively
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Create DataLoaders: here, we use mini-batch gradient descent, so need to specify the batch size
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

## Construct a binary classification model

In [31]:
class BinaryClassification(nn.Module):
    def __init__(self):
        super(BinaryClassification, self).__init__()
        #The first "30" specifies that the feature dimension is 30, and the second "1" specifies that this is binary classification
        self.fc = nn.Linear(30, 1) 
    
    def forward(self, x):
        y = self.fc(x)
        
        return y

## Set up some hyperparameters: use binary cross entropy loss, gradient descent with Adam optimizer, learning rate, and epochs

In [34]:
epochs = 50
learning_rate = 0.01
lossfunction = nn.BCEWithLogitsLoss() #Binary cross entropy loss for binary classification

#Instantiate the model from "BinaryClassification" class definition
model = BinaryClassification()

#Use Adam optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

print(model)

BinaryClassification(
  (fc): Linear(in_features=30, out_features=1, bias=True)
)


## Train the model using training data (X_train, y_train)

In [35]:
#Define the training function
def train(epoch, model, train_dataloader, optimizer, lossfunction):
    model.train()
    
    train_loss = 0.0
    train_total, train_correct = 0.0, 0.0 
    
    for X_batch, y_batch in train_dataloader:
        optimizer.zero_grad()

        #Get the predicted output
        predictions = model(X_batch)

        #Calculate the loss
        loss = lossfunction(predictions, y_batch)
        
        #Update the weights usning gradient descent with Adam optimizer
        loss.backward()
        optimizer.step()
        
        #Convert probabilities to binary predictions (if probability >= 0.5 reutrn positive prediction)
        train_predicted = (predictions >= 0.5).float()
        
        #Calculate the training statistics
        train_loss += loss.item()
        train_total += y_batch.size(0)
        train_correct += (train_predicted == y_batch).sum().item()

    print("epoch (%d): Train accuracy: %.4f, loss: %.3f" % (epoch, train_correct/train_total, train_loss/train_total))
    
    
#Train the model
for epoch in range(1, epochs + 1):
    train(epoch, model, train_dataloader, optimizer, lossfunction)

epoch (1): Train accuracy: 0.6308, loss: 0.603
epoch (2): Train accuracy: 0.7890, loss: 0.070
epoch (3): Train accuracy: 0.8747, loss: 0.035
epoch (4): Train accuracy: 0.8791, loss: 0.031
epoch (5): Train accuracy: 0.8703, loss: 0.030
epoch (6): Train accuracy: 0.8813, loss: 0.028
epoch (7): Train accuracy: 0.8615, loss: 0.032
epoch (8): Train accuracy: 0.8879, loss: 0.023
epoch (9): Train accuracy: 0.8901, loss: 0.023
epoch (10): Train accuracy: 0.8703, loss: 0.027
epoch (11): Train accuracy: 0.8923, loss: 0.020
epoch (12): Train accuracy: 0.8725, loss: 0.027
epoch (13): Train accuracy: 0.8923, loss: 0.026
epoch (14): Train accuracy: 0.9011, loss: 0.021
epoch (15): Train accuracy: 0.8769, loss: 0.021
epoch (16): Train accuracy: 0.9011, loss: 0.015
epoch (17): Train accuracy: 0.8879, loss: 0.020
epoch (18): Train accuracy: 0.8967, loss: 0.017
epoch (19): Train accuracy: 0.8989, loss: 0.018
epoch (20): Train accuracy: 0.9143, loss: 0.012
epoch (21): Train accuracy: 0.9187, loss: 0.015
e

## Test the model using test data (X_test, y_test)

In [37]:
#Define the test function
def test():
    model.eval()
    
    test_correct, test_total = 0.0, 0.0
    y_pred = []
    
    with torch.no_grad():
        for X_batch, y_batch in test_dataloader:
            predictions = model(X_batch)
            
            #Convert probabilities to binary predictions (if probability >= 0.5 reutrn positive prediction)
            test_predicted = (predictions >= 0.5).float()
        
            test_total += y_batch.size(0)
            test_correct += (test_predicted == y_batch).sum().item()
            
            y_pred += test_predicted.tolist()

    f1 = f1_score(y_test, y_pred)
    print('Test accuracy: %.4f, macro f1_score: %.4f' % (test_correct / test_total, f1))
    
    return y_pred

#Test the model
y_pred = test()

Test accuracy: 0.9737, macro f1_score: 0.9790


## Output the confusion matrix

In [41]:
confusion = confusion_matrix(y_test, y_pred)

print('Confusion Matrix\n')
print(confusion)

Confusion Matrix

[[41  2]
 [ 1 70]]


## Calculate accuracy and F1-score

In [42]:
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print('Accuracy: {:.2f}'.format(acc))
print('F1-score: {:.2f}'.format(f1))

Accuracy: 0.97
F1-score: 0.98
